In [1]:
# import os
# import pandas as pd

import os
import pandas as pd
import requests
from datetime import datetime
import json
import re

In [7]:
%%time
# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo'
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=demo'
web_data = requests.get(url)
web_data.encoding = 'utf-8'

web_data_text = web_data.text
json_acceptable_string = web_data_text.replace("'", "\"")
web_data_dict = json.loads(json_acceptable_string)

print(type(web_data_dict), len(web_data_dict), web_data_dict.keys())

del web_data, web_data_text, json_acceptable_string

meta_data_dict = web_data_dict["Meta Data"]
stock_data_dict = web_data_dict["Time Series (Daily)"]

del web_data_dict

print(meta_data_dict, "\n")

# print(type(stock_data_dict), stock_data_dict.keys())

# stock_time = list(stock_data_dict.keys())
# print(type(stock_time), stock_time[:3])

# stock_val = stock_data_dict.values()
# term = stock_data_dict["2018-05-24"]
# print(type(term), term, term["4. close"])

<class 'dict'> 2 dict_keys(['Meta Data', 'Time Series (Daily)'])
{'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'MSFT', '3. Last Refreshed': '2018-05-25', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'} 

CPU times: user 103 ms, sys: 8.49 ms, total: 112 ms
Wall time: 29.6 s


In [28]:
TRAINING_PATH = '../data/news/'
TESTING_PATH = '../data/news/'

In [29]:
old_categories = [dirname for dirname in os.listdir(TRAINING_PATH)]
categories= sorted(old_categories)
print(len(categories), str(categories))

895 ['2006-10-20', '2006-10-21', '2006-10-23', '2006-10-24', '2006-10-25', '2006-10-27', '2006-10-30', '2006-11-01', '2006-11-02', '2006-11-03', '2006-11-07', '2006-11-08', '2006-11-09', '2006-11-10', '2006-11-13', '2006-11-14', '2006-11-15', '2006-11-16', '2006-11-17', '2006-11-20', '2006-11-21', '2006-11-22', '2006-11-23', '2006-11-24', '2006-11-27', '2006-11-29', '2006-11-30', '2006-12-01', '2006-12-04', '2006-12-05', '2006-12-06', '2006-12-07', '2006-12-08', '2006-12-09', '2006-12-11', '2006-12-12', '2006-12-13', '2006-12-14', '2006-12-15', '2006-12-18', '2006-12-19', '2006-12-20', '2006-12-21', '2006-12-25', '2006-12-26', '2006-12-28', '2007-01-03', '2007-01-04', '2007-01-07', '2007-01-08', '2007-01-09', '2007-01-10', '2007-01-11', '2007-01-12', '2007-01-16', '2007-01-18', '2007-01-19', '2007-01-22', '2007-01-23', '2007-01-24', '2007-01-25', '2007-01-26', '2007-01-29', '2007-01-30', '2007-01-31', '2007-02-01', '2007-02-02', '2007-02-03', '2007-02-04', '2007-02-05', '2007-02-06', '

In [18]:
%%time
train_list = []

for category in categories:
    category_path = TRAINING_PATH + category + "/"
    
    for filename in os.listdir(category_path):
        filepath = category_path + filename
#         print(filepath)
        with open(filepath, encoding='utf-8') as file:
            try:
                words = file.read().strip().split('-- http://www.bloomberg.com')
                if words[0] == "":
                    pass
                else:
                    abs_words = words[0].split("\n")
                    if len(abs_words) > 2:
                        ari_title = abs_words[0]
                        ari_time = abs_words[2]
                        if category in stock_data_dict.keys():
                            price = stock_data_dict[category]["4. close"]
                            train_list.append([ari_time, ari_title, price])
            except ValueError:
                print (ValueError)
                break
#                 break

CPU times: user 2.36 s, sys: 531 ms, total: 2.89 s
Wall time: 2.91 s


In [19]:
train_df = pd.DataFrame(train_list, columns=["ari_time", "ari_title", "price"])
print("Shape:", train_df.shape)
train_df.sample(5)
print(train_df["ari_time"][0], train_df["ari_title"][0], "  [PRINT over] ")

Shape: (55483, 3)
-- 2006-10-20T20:16:16Z -- Inco's Net Soars on Higher Metal Prices, Breakup Fee   [PRINT over] 


In [20]:
%%time
train_df.to_pickle('train.pkl')

CPU times: user 34.1 ms, sys: 8.46 ms, total: 42.5 ms
Wall time: 42.3 ms


In [21]:
%time
pickle_df = pd.read_pickle('train.pkl')
# train_df.equals(pickle_df)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


In [27]:
print( pickle_df.shape, type(pickle_df))
# pickle_df[:3]["price"] = 10
print(pickle_df[:30]["price"])
# pickle_df.shape

(55483, 3) <class 'pandas.core.frame.DataFrame'>
0     28.4300
1     28.4500
2     28.4500
3     28.2800
4     28.2800
5     28.2800
6     28.2800
7     28.3100
8     28.3100
9     28.3400
10    28.3400
11    28.3400
12    28.3400
13    28.5300
14    28.5300
15    28.5300
16    28.8100
17    28.8100
18    28.8100
19    28.7700
20    28.7700
21    28.7300
22    28.7300
23    28.7300
24    28.9500
25    28.9500
26    28.9500
27    28.9500
28    28.9800
29    28.9800
Name: price, dtype: object
